This file aims to pre-process data to be implemented in MSPA
* recode background and foreground to MSPA
* reproject data if needed
* export as geotiff and byte/8-bit
* uses dask to parallelize processes 

In [ ]:
# Import Packages
import os
import re
import glob
from pathlib import Path
import rioxarray as rxr
import xarray as xr
from rasterio.enums import Resampling

In [2]:
# Initialize Dask cluster
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=4, threads_per_worker=1, memory_limit='8GB')
client = Client(cluster)
print(f"Dashboard: {client.dashboard_link}")

Dashboard: http://127.0.0.1:8787/status


In [ ]:
# Parameters
input_dir = "./code/data/tiff_to_use"
output_dir = "./code/data/tiff_P_geotiff_recoded"
tif_pattern = r'mosaic_(\d{4})\.tif'
background1 = 0 # original image background value
foreground1 = 1 # original image foreground value
background2 = 1 # target background value
foreground2 = 2 # target foreground value
source_crs = "EPSG:4326"
target_crs = "EPSG:29192"  # Set to None to skip reprojection

def process_files(input_dir, output_dir, source_crs, target_crs, background1, background2, foreground1, foreground2, tif_pattern):
    """
    This function processes all TIF files from a folder, recoding the background and foreground to suit MSPA input, reporject if needed,
    and exports image as GEOTIFF in 8-bit which is needed for MSPA. This is optimized to be used with Dask.

    Inputs:
    input_dir : filepath
        The path to the input folder containing the file or files.
    output_dir : filepath
        The path to the output folder where files will be exported.
    source_crs : str
        Contains the original data or "source" CRS that will be used for transforming. The format should be 'EPSG:XXXX'.
    target_crs : str
        Contains the target CRS that will be used for reprojecting. The format should be 'EPSG:XXXX'.
    background1 : int
        Contains the original images background value.
    background2 : int
        Contains the desired background value to be recoded to.
    foreground1 : int
        Contains the original images foreground value.
    foreground2 : int
        Contains the desired foreground value to be recoded to.
    tif_pattern : str
        Pattern of the tif files to extract the year from the file name.
    """
    # Read all TIFF files
    tif_files = glob.glob(os.path.join(input_dir, "*.tif"))
    print(f"Found {len(tif_files)} files to process")
    
    results = []
    for tif_file in tif_files:
        try:
            filename = os.path.basename(tif_file)
            print(f"Processing: {filename}")
            
            # Open with Dask chunking for lazy loading
            data = rxr.open_rasterio(
                tif_file,
                chunks={'band': 1, 'x': 1024, 'y': 1024}
            )

            # Prep files to be in xarray if no metadata
            if data.rio.crs is None:
                print(f"CRS missing in {filename}, setting to {source_crs}")
                data = data.rio.write_crs(source_crs)
            if not data.rio.transform():
                print(f"Transform missing in {filename}, setting from file")
                with rxr.open_rasterio(tif_file) as src:
                    data = data.rio.write_transform(src.rio.transform())
            
            # Recode data using Dask
            print(f"Recoding: {filename}")
            recoded = xr.where(data == foreground1, foreground2,
                      xr.where(data == background1, background2, 0)).astype('uint8')
            
            # 4. Preserve all metadata explicitly                       ##### there has to be a cleaner way to do this
            recoded = recoded.rio.write_crs(data.rio.crs)
            recoded = recoded.rio.write_transform(data.rio.transform())
            #recoded = recoded.rio.write_nodata(0) 

            # Reproject if target CRS is provided                       ##### rasterio warp vs rio? 
            print(f"Reprojecting: {filename}")
            if target_crs:
                recoded = recoded.rio.reproject(
                    target_crs,
                    resampling = Resampling.bilinear
                )
            
            # Generate output name                                      ##### NEED to change- if not reprojected = no _P_ in filename
            match = re.search(tif_pattern, filename)
            output_name = f"{match.group(1)}_P_recoded.tif" if match else f"{Path(tif_file).stem}_P_recoded.tif"
            output_path = os.path.join(output_dir, output_name)
            
            # Write with Dask parallelization
            print(f"Exporting: {filename}")
            recoded.rio.to_raster(
                output_path,
                dtype="uint8",
                compress='LZW',
                tiled=True,
                lock=True
            )
            print(f"Saved: {output_name}")
            results.append(True)

                                                                        ##### add a completed 1 out of 24 type thing
                                            # # Calculate success rate
                                            # success_count = sum(processing_results)
                                            # print(f"\nProcessed {success_count}/{len(processing_results)} files successfully")

        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
            results.append(False)
    
    return results

In [ ]:
# something like this:
for tif_file in input_dir.glob("*.tif"):
    # Extract year from filename (e.g., 'mosaic_1990.tif' -> '1990')
    match = re.search(r"mosaic_(\d{4})", tif_file.stem)
    if match:
        year = match.group(1)
        output_name = f"{year}_P_recoded.tif"
    else:
        output_name = f"{tif_file.stem}_P_recoded.tif"
with rasterio.open(tif_file) as src:
    data = src.read(1)
    recoded = recode_for_mspa(data)
    transform, width, height = rasterio.warp.calculate_default_transform(
        src.crs, target_crs, src.width, src.height, *src.bounds)
    profile = src.profile.copy()
    profile.update({
        'driver': 'GTiff',
        'dtype': 'uint8',
        'crs': target_crs,
        'transform': transform,
        'width': width,
        'height': height,
        'count': 1,
        'compress': 'lzw'
    })
    reprojected = np.empty((height, width), dtype=np.uint8)
    rasterio.warp.reproject(
        source=recoded,
        destination=reprojected,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=transform,
        dst_crs=target_crs,
        resampling=Resampling.bilinear
    )
    output_path = output_dir / output_name
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(reprojected, 1)

In [ ]:
# Execute the processing
processed = process_files(
    input_dir=input_dir, 
    output_dir=output_dir,
    source_crs=source_crs,
    target_crs=target_crs,
    background1=background1,
    background2=background2,
    foreground1=foreground1,
    foreground2=foreground2,
    tif_pattern=tif_pattern
)
    
# Cleanup
#client.close()
#cluster.close()

Found 24 files to process
Processing: mosaic_1991.tif
Recoding: mosaic_1991.tif
Reprojecting: mosaic_1991.tif
Exporting: mosaic_1991.tif
Saved: 1991_P_recoded.tif
Processing: mosaic_1992.tif
Recoding: mosaic_1992.tif
Reprojecting: mosaic_1992.tif
Exporting: mosaic_1992.tif
Saved: 1992_P_recoded.tif
Processing: mosaic_1993.tif
Recoding: mosaic_1993.tif
Reprojecting: mosaic_1993.tif
Exporting: mosaic_1993.tif
Saved: 1993_P_recoded.tif
Processing: mosaic_1994.tif
Recoding: mosaic_1994.tif
Reprojecting: mosaic_1994.tif
Exporting: mosaic_1994.tif
Saved: 1994_P_recoded.tif
Processing: mosaic_1996.tif
Recoding: mosaic_1996.tif
Reprojecting: mosaic_1996.tif
Exporting: mosaic_1996.tif
Saved: 1996_P_recoded.tif
Processing: mosaic_1997.tif
Recoding: mosaic_1997.tif
Reprojecting: mosaic_1997.tif
Exporting: mosaic_1997.tif
Exporting: mosaic_1999.tif
Saved: 1999_P_recoded.tif
Processing: mosaic_2001.tif
Recoding: mosaic_2001.tif
Reprojecting: mosaic_2001.tif
Exporting: mosaic_2001.tif
Saved: 2001_P_